In [26]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder,StandardScaler,LabelBinarizer,MinMaxScaler, PolynomialFeatures
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,f1_score,roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [27]:
df = pd.read_csv('../2.DataPreProcessing/preprocessed_online_food_dataset.csv')
df.head()

,Age,Gender,Marital Status,Occupation,Monthly Income,Educational Qualifications,Family size,latitude,longitude,Pin code,Output,Feedback
0,20,Female,Single,Student,No Income,Post Graduate,4,12.9766,77.5993,560001,Yes,Positive
1,24,Female,Single,Student,Below Rs.10000,Graduate,3,12.9770,77.5773,560009,Yes,Positive
2,22,Male,Single,Student,Below Rs.10000,Post Graduate,3,12.9551,77.6593,560017,Yes,Negative
3,22,Female,Single,Student,No Income,Graduate,6,12.9473,77.5616,560019,Yes,Positive
4,22,Male,Single,Student,Below Rs.10000,Post Graduate,4,12.9850,77.5533,560010,Yes,Positive


In [28]:
#Converting all categorical to numerical values

In [29]:
df.dtypes

Age                             int64
Gender                         object
Marital Status                 object
Occupation                     object
Monthly Income                 object
Educational Qualifications     object
Family size                     int64
latitude                      float64
longitude                     float64
Pin code                        int64
Output                         object
Feedback                       object
dtype: object

In [30]:
print('Gender Counts: ', df['Gender'].value_counts())
print('-----------')
print('Marital Status Counts: ', df['Marital Status'].value_counts())
print('-----------')
print('Occupation counts: ', df['Occupation'].value_counts())
print('-----------')
print('Monthly Income counts: ', df['Monthly Income'].value_counts())
print('-----------')
print('Educational Qualifications counts: ', df['Educational Qualifications'].value_counts())
print('-----------')
print('Feedback counts: ', df['Feedback'].value_counts())

Gender Counts:  Gender
Male      222
Female    166
Name: count, dtype: int64
-----------
Marital Status Counts:  Marital Status
Single               268
Married              108
Prefer not to say     12
Name: count, dtype: int64
-----------
Occupation counts:  Occupation
Student           207
Employee          118
Self Employeed     54
House wife          9
Name: count, dtype: int64
-----------
Monthly Income counts:  Monthly Income
No Income          187
25001 to 50000      69
More than 50000     62
10001 to 25000      45
Below Rs.10000      25
Name: count, dtype: int64
-----------
Educational Qualifications counts:  Educational Qualifications
Graduate         177
Post Graduate    174
Ph.D              23
School            12
Uneducated         2
Name: count, dtype: int64
-----------
Feedback counts:  Feedback
Positive     317
Negative      71
Name: count, dtype: int64


In [32]:
df['Gender'].unique()

array(['Female', 'Male'], dtype=object)

In [6]:
df['Gender'] = df["Gender"].map({"Male": 1, "Female": 0})
df['Marital Status'] = df['Marital Status'].map({'Prefer not to say': 0, 'Single': 1, 'Married': 2})
df['Occupation'] = df['Occupation'].map({"Student": 1, "Employee": 2, "Self Employeed": 3, "House wife": 4})
df["Educational Qualifications"] = df["Educational Qualifications"].map({"Graduate": 1, "Post Graduate": 2, "Ph.D": 3, "School": 4, "Uneducated": 5})
df["Monthly Income"] = df["Monthly Income"].map({"No Income": 0, "25001 to 50000": 5000, "More than 50000": 7000, "10001 to 25000": 25000, "Below Rs.10000": 10000})
df["Feedback"] = df["Feedback"].map({"Positive": 1, "Negative ": 0})

In [7]:
df.head()

,Age,Gender,Marital Status,Occupation,Monthly Income,Educational Qualifications,Family size,latitude,longitude,Pin code,Output,Feedback
0,20,0,1,1,0,2,4,12.9766,77.5993,560001,Yes,1
1,24,0,1,1,10000,1,3,12.9770,77.5773,560009,Yes,1
2,22,1,1,1,10000,2,3,12.9551,77.6593,560017,Yes,0
3,22,0,1,1,0,1,6,12.9473,77.5616,560019,Yes,1
4,22,1,1,1,10000,2,4,12.9850,77.5533,560010,Yes,1


In [8]:
#Preparing The Model

In [9]:
X = df.drop(['Output', 'latitude', 'longitude', 'Pin code'], axis=1)
y = df['Output']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [10]:
# Initialize and train a random forest classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Make predictions on the test set
y_pred = clf.predict(X_test)

# Evaluate model performance
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Feature transformation (adding polynomial features)
poly = PolynomialFeatures(degree=2)
X_train_poly = poly.fit_transform(X_train_scaled)
X_test_poly = poly.transform(X_test_scaled)

# Feature selection
selector = SelectKBest(score_func=f_classif, k=2)
X_train_selected = selector.fit_transform(X_train_poly, y_train)
X_test_selected = selector.transform(X_test_poly)

# Train a new model using the transformed and selected features
clf.fit(X_train_selected, y_train)
y_pred_selected = clf.predict(X_test_selected)

# Evaluate model performance
accuracy_selected = accuracy_score(y_test, y_pred_selected)

print("Accuracy with feature engineering:", accuracy_selected)

Accuracy: 0.8974358974358975
Accuracy with feature engineering: 0.8547008547008547


C:\Users\HP\anaconda3\envs\aiml\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: UserWarning: Features [0] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
C:\Users\HP\anaconda3\envs\aiml\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


In [11]:
from sklearn.metrics import roc_curve, auc
models = {
    'Logistic Regression': LogisticRegression(random_state=42),
    'Random Forest Classifier': RandomForestClassifier(random_state=42),
    'K-Nearest Neighbors Classifie': KNeighborsClassifier(),
    'Support Vector Machine' : SVC()
}
lb = LabelBinarizer()
lb.fit(y_train)
for name, model in models.items():
    print(f"Model: {name}")
    
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(classification_report(y_test, y_pred),'\n')
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred),'\n')
    
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy Score: {accuracy} \n")
    
    f1 = f1_score(lb.transform(y_test), lb.transform(y_pred), pos_label=1)
    print(f"F1 Score: {f1}")
    
   
    print("-" * 50,'\n')

Model: Logistic Regression
              precision    recall  f1-score   support

          No       0.63      0.55      0.59        22
         Yes       0.90      0.93      0.91        95

    accuracy                           0.85       117
   macro avg       0.76      0.74      0.75       117
weighted avg       0.85      0.85      0.85       117
 

Confusion Matrix:
[[12 10]
 [ 7 88]] 

Accuracy Score: 0.8547008547008547 

F1 Score: 0.911917098445596
-------------------------------------------------- 

Model: Random Forest Classifier
              precision    recall  f1-score   support

          No       0.81      0.59      0.68        22
         Yes       0.91      0.97      0.94        95

    accuracy                           0.90       117
   macro avg       0.86      0.78      0.81       117
weighted avg       0.89      0.90      0.89       117
 

Confusion Matrix:
[[13  9]
 [ 3 92]] 

Accuracy Score: 0.8974358974358975 

F1 Score: 0.9387755102040817
---------------------

C:\Users\HP\anaconda3\envs\aiml\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


F1 Score: 0.9238578680203046
-------------------------------------------------- 

Model: Support Vector Machine
              precision    recall  f1-score   support

          No       0.00      0.00      0.00        22
         Yes       0.81      1.00      0.90        95

    accuracy                           0.81       117
   macro avg       0.41      0.50      0.45       117
weighted avg       0.66      0.81      0.73       117
 

Confusion Matrix:
[[ 0 22]
 [ 0 95]] 

Accuracy Score: 0.811965811965812 

F1 Score: 0.8962264150943396
-------------------------------------------------- 



C:\Users\HP\anaconda3\envs\aiml\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\HP\anaconda3\envs\aiml\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\HP\anaconda3\envs\aiml\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [12]:
#Grid Search Cross Validation

In [13]:
from sklearn.model_selection import GridSearchCV
grid = {"n_estimators":np.arange(1,10)}
rf= RandomForestClassifier()
rf_cv =  GridSearchCV(rf,grid,cv = 10)
rf_cv.fit(X_train,y_train)

GridSearchCV(cv=10, estimator=RandomForestClassifier(),
             param_grid={'n_estimators': array([1, 2, 3, 4, 5, 6, 7, 8, 9])})

In [14]:
print("tuned hypermeter Random:",rf_cv.best_params_)
print("best accuracy to tuned hypermeter:",rf_cv.best_score_)

tuned hypermeter Random: {'n_estimators': 8}
best accuracy to tuned hypermeter: 0.8892857142857142


In [33]:
import pickle
from sklearn.metrics import mean_squared_error
modelFinal = RandomForestClassifier(random_state=42)
modelFinal.fit(X_train, y_train)

# Make predictions
y_predfinal = modelFinal.predict(X_test)

print(classification_report(y_test, y_predfinal),'\n')
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_predfinal),'\n')

accuracy = accuracy_score(y_test, y_predfinal)
print(f"Accuracy Score: {accuracy} \n")

f1 = f1_score(lb.transform(y_test), lb.transform(y_pred), pos_label=1)
print(f"F1 Score: {f1}")

# Save the model using pickle
with open("Online_Food_Order_Prediction.pkl", "wb") as f:
    pickle.dump(modelFinal, f)
print("Model saved successfully!")

              precision    recall  f1-score   support

          No       0.81      0.59      0.68        22
         Yes       0.91      0.97      0.94        95

    accuracy                           0.90       117
   macro avg       0.86      0.78      0.81       117
weighted avg       0.89      0.90      0.89       117
 

Confusion Matrix:
[[13  9]
 [ 3 92]] 

Accuracy Score: 0.8974358974358975 

F1 Score: 0.8962264150943396
Model saved successfully!


In [34]:
# Later, you can load the model using pickle
with open("Online_Food_Order_Prediction.pkl", "rb") as f:
    loaded_model = pickle.load(f)

In [24]:
#Age -> 21
#Gender -> Male [1]
#Marital Status -> Single [1]
#Occupation -> Student [1]
#Monthly Income -> No Income [0]
#Educational Qualification -> Graduate [1]
#Family Size -> 1
#Feedback -> No [0]
result = loaded_model.predict([[21, 1, 2, 1, 0, 1, 1, 0]])
print(result)

['No']


C:\Users\HP\anaconda3\envs\aiml\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [35]:
#Age -> 21
#Gender -> Male [1]
#Marital Status -> Single [1]
#Occupation -> Student [1]
#Monthly Income -> No Income [0]
#Educational Qualification -> Graduate [1]
#Family Size -> 3
#Feedback -> Yes [1]
result = loaded_model.predict([[21, 1, 1, 1, 0, 1, 3, 1]])
print(result)

['Yes']


C:\Users\HP\anaconda3\envs\aiml\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
